# Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this notebook tutorial on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078) This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

The main bottleneck of seq2seq model is the context vector ``z`` from the output of ``encoder``. In the basic setting, ``z`` only serves the input for the first cell of ``decoder``, therefore ``z`` is froced to encode the whole source sequence with high quality, which is difficult for long sequence. To allevate the restriction, the new model improves the ``decoder`` by connecting ``z`` with each cell in the ``decoder``. The details are illustrated later.

## 1. Prepare Data

All of the data preparation will be (almost) the same as last time, so we'll very briefly detail what each code block does. See the previous notebook for a recap.

We'll import PyTorch, TorchText, spaCy and a few standard modules.

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import time
import math

In [0]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

Initiate English and German Spacy models.

In [3]:
!python -m spacy download en
!python -m spacy download de
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


Previously we reversed the source (German) sentence, however in the 
paper we are implementing they don't do this, so neither will we.

In [0]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [0]:
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

Load the data and print out some example

In [0]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [7]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [0]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

Finally, define the `device` and create our iterators.

In [0]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = DEVICE
)

## Building the Model

### Encoder
Different than previous encoder, here multi-layer LSTM are swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that *dropout is used between each layer of a multi-layered RNN*. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM).

The rest of the encoder should be very familar from the last tutorial, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](../figs/seq2seq5.png)

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim 
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        #src = [src_len, batch_size]
        
        embedded = self.dropout(self.embedding(src))
        # embedded = [src_len, batch_size, emb_dim]

        output, hidden = self.rnn(embedded)
        # output = [src_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers*n_directions, batch_size, hid_dim]

        ###outpus are always from the top layer, hidden is the context vector z
        return hidden


### Decoder
The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression, in two ways:

* Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder.

* Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](../figs/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.



How do these two changes reduce the demand of information compression in decoder? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it. Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        self.fc_out = nn.Linear(emb_dim + 2 * hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, context):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]

        input = input.unsqueeze(0)
        # input = [1, batch_size]

        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]

        emb_con = torch.cat((embedded, context), dim=2) ## concatenate input and context vector
        # emb_con = [1, batch_size, emb_dim + hid_dim]

        output, hidden = self.rnn(emb_con, hidden)
        # output = [1, batch size, hid dim]
        # hidden = [1, batch size, hid dim]

        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)
        # output = [batch_size, emb dim + hid dim * 2]

        prediction = self.fc_out(output)
        # prediction = [batch_size, output_dim]

        return prediction, hidden



### Seq2Seq Model
Putting the encoder and decoder together, we get:

![](../figs/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to generate a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - generating a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

We also initialize our parameters as the paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim

    def forward(self, src, trg, teacher_forcing_ratio=0.5):

        #src = [src_len, batch_size]
        #trg = [trg_len, batch_size]

        trg_len, batch_size = trg.shape
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # last hidden state of the encoder is the context vector
        context = self.encoder(src)

        # context also used as the initial hidden state of the decoder
        hidden = context

        # first input to the decoder is the <sos> tokens
        input = trg[0, :]

        for t in range(1, trg_len):
            # insert input embeddings, context vector, and previous hidden state
            # emit output (prediction) and current hidden state
            output, hidden = self.decoder(input, hidden, context)

            # add the current prediction to the storing outputs
            outputs[t] = output 

            # decide if use teacher_forcing or not
            teacher_force = random.random() < teacher_forcing_ratio 

            # get the top1 prediction
            top = output.argmax(1)

            input = trg[t] if teacher_force else top 

        return outputs

    def init_weights(self):
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.01)


## Train the Model
The rest of this tutorial is very similar to the previous one. 



### Initialization
We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = DEC_DROPOUT = 0.5

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

In [0]:
model.init_weights()

In [16]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer in the decoder. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [17]:
def count_parameters(model):
    for name, para in model.named_parameters():
        print(name, para.requires_grad, para.data.shape)

    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

encoder.embedding.weight True torch.Size([7855, 256])
encoder.rnn.weight_ih_l0 True torch.Size([1536, 256])
encoder.rnn.weight_hh_l0 True torch.Size([1536, 512])
encoder.rnn.bias_ih_l0 True torch.Size([1536])
encoder.rnn.bias_hh_l0 True torch.Size([1536])
decoder.embedding.weight True torch.Size([5893, 256])
decoder.rnn.weight_ih_l0 True torch.Size([1536, 768])
decoder.rnn.weight_hh_l0 True torch.Size([1536, 512])
decoder.rnn.bias_ih_l0 True torch.Size([1536])
decoder.rnn.bias_hh_l0 True torch.Size([1536])
decoder.fc_out.weight True torch.Size([5893, 1280])
decoder.fc_out.bias True torch.Size([5893])
The model has 14,220,293 trainable parameters


Set up the optimizer and criterion

In [0]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

### train and test


In [0]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src 
        trg = batch.trg 

        optimizer.zero_grad()
        output = model(src, trg)

        # trg = [trg_len, batch_size]
        # output = [trg_len, batch_size, output_dim]

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim) ### skip the <sos> tag
        trg = trg[1:].view(-1)
        #trg = [(trg_len - 1) * batch_size]
        #output = [(trg_len - 1) * batch_size, output_dim]

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss += loss 

    return epoch_loss / len(iterator)






In [0]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Start the training loop

In [22]:
N_EPOCHS = 10
CLIP = 1 

best_valid_loss = float('inf')
best_model = "seq2seq.pt"

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), best_model)

    print(f'Epoch: {epoch} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 0 | Time: 0m 28s
	Train Loss: 5.080 | Train PPL: 160.808
	 Val. Loss: 5.111 |  Val. PPL: 165.866
Epoch: 1 | Time: 0m 28s
	Train Loss: 4.394 | Train PPL:  80.936
	 Val. Loss: 5.137 |  Val. PPL: 170.138
Epoch: 2 | Time: 0m 28s
	Train Loss: 4.069 | Train PPL:  58.515
	 Val. Loss: 4.782 |  Val. PPL: 119.365
Epoch: 3 | Time: 0m 28s
	Train Loss: 3.726 | Train PPL:  41.527
	 Val. Loss: 4.445 |  Val. PPL:  85.167
Epoch: 4 | Time: 0m 27s
	Train Loss: 3.410 | Train PPL:  30.261
	 Val. Loss: 4.131 |  Val. PPL:  62.213
Epoch: 5 | Time: 0m 28s
	Train Loss: 3.127 | Train PPL:  22.814
	 Val. Loss: 3.967 |  Val. PPL:  52.803
Epoch: 6 | Time: 0m 28s
	Train Loss: 2.888 | Train PPL:  17.952
	 Val. Loss: 3.776 |  Val. PPL:  43.635
Epoch: 7 | Time: 0m 28s
	Train Loss: 2.644 | Train PPL:  14.064
	 Val. Loss: 3.723 |  Val. PPL:  41.373
Epoch: 8 | Time: 0m 28s
	Train Loss: 2.426 | Train PPL:  11.315
	 Val. Loss: 3.698 |  Val. PPL:  40.373
Epoch: 9 | Time: 0m 28s
	Train Loss: 2.256 | Train PPL:   9.542


Finally, test the model on test dataset using "best" model

In [23]:
model.load_state_dict(torch.load(best_model))
test_loss = evaluate(model, test_iterator, criterion)
print(f"Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} ")

Test Loss: 3.608 | Test PPL:  36.903 


Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.

## References:
https://github.com/bentrevett/pytorch-seq2seq/blob/master/2%20-%20Learning%20Phrase%20Representations%20using%20RNN%20Encoder-Decoder%20for%20Statistical%20Machine%20Translation.ipynb